In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import plotly.express as px
import pycountry
import altair as alt

In [3]:
df_terror = pd.read_csv('data/gtd_1970_2021.csv')
# Print columns and row count
print(df_terror.columns)
print(df_terror.shape)

C:\Users\Johnny Chen\AppData\Local\Temp\ipykernel_7204\2926398536.py:1: DtypeWarning: Columns (4,31,33,54,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df_terror = pd.read_csv('data/gtd_1970_2021.csv')


Index(['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended',
       'resolution', 'country', 'country_txt', 'region',
       ...
       'addnotes', 'scite1', 'scite2', 'scite3', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'],
      dtype='object', length=135)
(214666, 135)


In [4]:
df_refugee = pd.read_csv('data/population.csv')
# Print columns and row count
print(df_refugee.columns)
print(df_refugee.shape)

Index(['Year', 'Country of origin', 'Country of origin (ISO)',
       'Country of asylum', 'Country of asylum (ISO)',
       'Refugees under UNHCR's mandate', 'Asylum-seekers',
       'IDPs of concern to UNHCR',
       'Other people in need of international protection', 'Stateless persons',
       'Host Community', 'Others of concern'],
      dtype='object')
(113209, 12)


In [5]:
s1 = set(df_terror['country_txt'])
s2 = set(df_refugee['Country of origin'])

# Creat ISO3 dictionary for s1
iso3_dict = {}
for country in s1:
    try:
        iso3_dict[country] = pycountry.countries.search_fuzzy(country)[0].alpha_3
    except:
        print(country)
        iso3_dict[country] = "Unknown"

#  Map ISO3 to country_txt
df_terror['iso3'] = df_terror['country_txt'].map(iso3_dict)
# Drop Unknown
df_terror = df_terror[df_terror['iso3'] != 'Unknown']

International
Czechoslovakia
North Yemen
Ivory Coast
Swaziland
West Germany (FRG)
People's Republic of the Congo
Yugoslavia
Macau
Democratic Republic of the Congo
New Hebrides
Soviet Union
St. Kitts and Nevis
Rhodesia
Bosnia-Herzegovina
West Bank and Gaza Strip
St. Lucia
South Yemen
East Timor
Serbia-Montenegro
Laos
East Germany (GDR)


In [6]:
s1 = set(df_terror['iso3'])
s2 = set(df_refugee['Country of origin (ISO)'])

# Difference between s1 and s2
print(len(s1))
print(len(s2))
print(s1 - s2)
print(s2 - s1)
# Only keep the intersection
df_refugee = df_refugee[df_refugee['Country of origin (ISO)'].isin(s1)]
df_refugee = df_refugee[df_refugee['Country of origin (ISO)'] != 'Unknown']
df_terror = df_terror[df_terror['iso3'].isin(s2)]
df_terror = df_terror[df_terror['iso3'] != 'Unknown']

print(df_terror.shape)
print(df_refugee.shape)

177
214
{'FLK', 'PYF', 'WLF', 'TWN'}
{'OMN', 'PLW', 'FSM', 'NIU', 'WSM', 'TIB', 'GLP', 'AIA', 'VCT', 'PSE', 'TCA', 'CYM', 'BMU', 'MCO', 'UNK', 'LCA', 'SMR', 'STP', 'BIH', 'CUW', 'MHL', 'MTQ', 'CIV', 'KIR', 'LAO', 'NER', 'GIB', 'PRI', 'TLS', 'TUV', 'XXA', 'MAC', 'CPV', 'KNA', 'SWZ', 'LIE', 'ABW', 'COD', 'NRU', 'TON', 'MNG'}
(208936, 136)
(99874, 12)


In [7]:
# For refugee, groupby Country of origin and year, then sum the population
df_refugee_country_origin = df_refugee.groupby(['Country of origin (ISO)', 'Year']).sum().reset_index()
df_refugee_country_dest = df_refugee.groupby(['Country of asylum (ISO)', 'Year']).sum().reset_index()

# Take the difference between the years
#df_refugee_country['Refugees under UNHCR\'s mandate'] = df_refugee_country.groupby('Country of origin (ISO)')['Refugees under UNHCR\'s mandate'].diff()
#print(df_refugee_country.head())
display(df_refugee_country_origin)
display(df_refugee_country_dest)

# Plot the refugee population by year for each country
fig = px.line(df_refugee_country_origin, x="Year", y="Refugees under UNHCR's mandate", color='Country of origin (ISO)')
fig.show()

# Take the year of 2021
df_refugee_country_year = df_refugee_country_origin[df_refugee_country_origin['Year'] == 2021]

fig = px.choropleth(df_refugee_country_year, locations="Country of origin (ISO)",
                    color="Refugees under UNHCR's mandate", # lifeExp is a column of gapminder
                    hover_name="Country of origin (ISO)", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

C:\Users\Johnny Chen\AppData\Local\Temp\ipykernel_7204\3675461347.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_refugee_country_origin = df_refugee.groupby(['Country of origin (ISO)', 'Year']).sum().reset_index()
C:\Users\Johnny Chen\AppData\Local\Temp\ipykernel_7204\3675461347.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_refugee_country_dest = df_refugee.groupby(['Country of asylum (ISO)', 'Year']).sum().reset_index()


,Country of origin (ISO),Year,Refugees under UNHCR's mandate,Asylum-seekers,IDPs of concern to UNHCR,Stateless persons,Others of concern
0,AFG,1979,500000,0,0,0,0
1,AFG,1980,1734921,0,0,0,0
2,AFG,1981,3879984,0,0,0,0
3,AFG,1982,4488214,0,0,0,0
4,AFG,1983,4712735,0,0,0,0
...,...,...,...,...,...,...,...
5894,ZWE,2017,17420,21760,0,0,169
5895,ZWE,2018,15618,17121,0,0,110
5896,ZWE,2019,10045,16177,270000,0,121
5897,ZWE,2020,8575,14614,0,0,132


,Country of asylum (ISO),Year,Refugees under UNHCR's mandate,Asylum-seekers,IDPs of concern to UNHCR,Stateless persons,Others of concern
0,ABW,2011,0,5,0,0,0
1,ABW,2012,0,5,0,0,0
2,ABW,2013,0,5,0,0,0
3,ABW,2014,0,5,0,0,0
4,ABW,2016,0,5,0,0,0
...,...,...,...,...,...,...,...
5723,ZWE,2017,1260,8147,0,0,453
5724,ZWE,2018,1295,8236,0,0,266
5725,ZWE,2019,1461,8535,270000,0,321
5726,ZWE,2020,1492,8649,0,0,309


In [8]:
# For terroism, groupby Country of origin and year, then count the number of attacks
df_terror_country = df_terror.groupby(['iso3', 'iyear']).count().reset_index()
print(df_terror_country.head())

# Plot the terroism count by year for each country
fig = px.line(df_terror_country, x="iyear", y="eventid", color='iso3')
fig.show()

# Take the year of 2021
df_terror_country_year = df_terror_country[df_terror_country['iyear'] == 2021]

fig = px.choropleth(df_terror_country_year, locations="iso3",
                    color="eventid", # lifeExp is a column of gapminder
                    hover_name="iso3", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

  iso3  iyear  eventid  imonth  iday  approxdate  extended  resolution  \
0  AFG   1973        1       1     1           0         1           0   
1  AFG   1979        3       3     3           0         3           0   
2  AFG   1987        1       1     1           0         1           0   
3  AFG   1988       11      11    11           0        11           0   
4  AFG   1989       10      10    10           0        10           0   

   country  country_txt  ...  addnotes  scite1  scite2  scite3  dbsource  \
0        1            1  ...         0       0       0       0         1   
1        3            3  ...         0       0       0       0         3   
2        1            1  ...         0       0       0       0         1   
3       11           11  ...         0       0       0       0        11   
4       10           10  ...         0       0       0       0        10   

   INT_LOG  INT_IDEO  INT_MISC  INT_ANY  related  
0        1         1         1        1        

In [9]:
# Merge the two dataframes
df_terror_refugee_origin = pd.merge(df_terror_country, df_refugee_country_origin, left_on=['iso3', 'iyear'], right_on=['Country of origin (ISO)', 'Year'])
df_terror_refugee_origin = df_terror_refugee_origin.rename(columns={'eventid': 'Attack Count', 'Refugees under UNHCR\'s mandate': 'Refugee Count (origin)'})

df_terror_refugee_dest = pd.merge(df_terror_country, df_refugee_country_dest, left_on=['iso3', 'iyear'], right_on=['Country of asylum (ISO)', 'Year'])
df_terror_refugee_dest = df_terror_refugee_dest.rename(columns={'eventid': 'Attack Count', 'Refugees under UNHCR\'s mandate': 'Refugee Count (destination)'})

# Only keep the eventid, population, and iso3 columns
df_terror_refugee_origin = df_terror_refugee_origin[['Attack Count', 'Refugee Count (origin)', 'Country of origin (ISO)', 'iso3', 'iyear']]
df_terror_refugee_dest = df_terror_refugee_dest[['Attack Count', 'Refugee Count (destination)', 'Country of asylum (ISO)', 'iso3', 'iyear']]

#df_terror_refugee = df_terror_refugee.groupby('iso3').corr().reset_index()
display(df_terror_refugee_origin)
display(df_terror_refugee_dest)


,Attack Count,Refugee Count (origin),Country of origin (ISO),iso3,iyear
0,3,500000,AFG,AFG,1979
1,1,5511740,AFG,AFG,1987
2,11,5622982,AFG,AFG,1988
3,10,5643989,AFG,AFG,1989
4,2,6339095,AFG,AFG,1990
...,...,...,...,...,...
2936,1,22478,ZWE,ZWE,2014
2937,3,17420,ZWE,ZWE,2017
2938,2,15618,ZWE,ZWE,2018
2939,2,10045,ZWE,ZWE,2019


,Attack Count,Refugee Count (destination),Country of asylum (ISO),iso3,iyear
0,36,60000,AFG,AFG,1992
1,9,19134,AFG,AFG,1994
2,6,19605,AFG,AFG,1995
3,4,18779,AFG,AFG,1996
4,1,5,AFG,AFG,1997
...,...,...,...,...,...
2885,1,1201,ZWE,ZWE,2014
2886,3,1260,ZWE,ZWE,2017
2887,2,1295,ZWE,ZWE,2018
2888,2,1461,ZWE,ZWE,2019


In [10]:
# Create column that maps iso3 to full country name
df_terror_refugee_dest["country"] = df_terror_refugee_dest["iso3"].apply(lambda x: pycountry.countries.get(alpha_3=x).name)
df_terror_refugee_origin["country"] = df_terror_refugee_origin["iso3"].apply(lambda x: pycountry.countries.get(alpha_3=x).name)

# Save the dataframe
df_terror_refugee_origin.to_csv('data/terror_refugee_origin.csv', index=False)
df_terror_refugee_dest.to_csv('data/terror_refugee_dest.csv', index=False)

In [11]:
iso_code = 'AFG'

In [12]:
# Choose a country
df_terror_refugee_origin = df_terror_refugee_origin[df_terror_refugee_origin['iso3'] == iso_code]

# Plot the attack count and refugee population by year with altair
# Use two y-axis and show legend
origin_base = alt.Chart(df_terror_refugee_origin).encode(
    alt.X('iyear:Q', axis=alt.Axis(title='Year')),
)

origin_line1 = origin_base.mark_line(color='red').encode(
    alt.Y('Attack Count:Q', axis=alt.Axis(title='Attack Count')).axis(titleColor='red')
)

origin_line2 = origin_base.mark_line(color='green').encode(
    alt.Y('Refugee Count (origin):Q', axis=alt.Axis(title='Refugee Count (origin)')).axis(titleColor='green')
)

alt.layer(origin_line1, origin_line2).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

In [13]:
# Choose a country
df_terror_refugee_dest = df_terror_refugee_dest[df_terror_refugee_dest['iso3'] == iso_code]

dest_base = alt.Chart(df_terror_refugee_dest).encode(
    alt.X('iyear:Q', axis=alt.Axis(title='Year')),
)

dest_line1 = dest_base.mark_line(color='red').encode(
    alt.Y('Attack Count:Q', axis=alt.Axis(title='Attack Count')).axis(titleColor='red')
)

dest_line2 = dest_base.mark_line(color='blue').encode(
    alt.Y('Refugee Count (destination):Q', axis=alt.Axis(title='Refugee Count (destination)')).axis(titleColor='blue')
)

alt.layer(dest_line1, dest_line2).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)